In [10]:
import requests
import pandas as pd
import pymysql
import os
from dotenv import load_dotenv

CONFIG = {
    'github_token': os.getenv('GITHUB_TOKEN'),
    'host': os.getenv('DB_HOST'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_NAME'),
    'charset': os.getenv('DB_CHARSET')
}

In [12]:
# GitHub API 토큰 설정 (GitHub에서 Personal Access Token 생성 필요)
github_token = CONFIG['github_token']  # 여기에 토큰 입력
headers = {'Authorization': f'token {github_token}'}

# 검색 키워드 설정
keyword = ["robotics", "ROS", "robot arm", "robot", "amr"]

# GitHub API 엔드포인트
serch_url = 'https://api.github.com/search/repositories'

In [4]:
# 키워드 해당되는 repo 가져오는 함수
def fetch_repos(query, sort='stars', order='desc', per_page=100):
    all_repos = []
    page = 1
    while True:
        params = {
            'q': f'{query} stars:>=100',  # star 수 100개 이상 조건 추가
            'sort': sort,
            'order': order,
            'per_page': per_page,
            'page': page
        }
        response = requests.get(serch_url, headers=headers, params=params)
        if response.status_code == 200:
            repos = response.json()['items']
            if not repos:
                break
            all_repos.extend(repos)
            page += 1
            # GitHub API rate limit을 고려해 최대 1000개로 제한 (10페이지)
            if page > 10:
                break
        else:
            print(f"Error: {response.status_code} - {response.text}")
            break
    return all_repos

In [5]:
# repo 메타데이터 추출하는 함수
def get_repo_details(repo):
    return {
		'id': repo['id'],
        'full_name': repo['full_name'],
        'stargazers_count': repo['stargazers_count'],
        'forks_count': repo['forks_count'],
        'language': repo['language'],
        'created_at': repo['created_at'],
        'updated_at': repo['updated_at'],
        'open_issues_count': repo['open_issues_count'],
        'topics': repo.get('topics', []),
        'license': repo['license']['spdx_id'] if repo.get('license') else None,
        'owner': repo['owner']['login'],
        'description': repo['description'],
		'is_fork' : repo['fork']
    }

In [6]:
# 추출한 데이터 sql로 저장하는 함수
def save_to_mysql(repos_list):
    """MySQL RDS에 GitHub 레포 데이터 저장"""
    try:
        # MySQL 연결
        connection = pymysql.connect(
            host=CONFIG['host'],
            user=CONFIG['user'],
            password=CONFIG['password'],
            database=CONFIG['database'],
            charset=CONFIG['charset'],
            cursorclass=pymysql.cursors.DictCursor
        )
        
        with connection.cursor() as cursor:
            # 테이블 생성 (존재하지 않을 경우)
            create_table_query = """
            CREATE TABLE IF NOT EXISTS github_repos (
                id BIGINT PRIMARY KEY,
                full_name VARCHAR(255) UNIQUE NOT NULL,
                stargazers_count INT DEFAULT 0,
                forks_count INT DEFAULT 0,
                language VARCHAR(100),
                created_at DATETIME,
                updated_at DATETIME,
                open_issues_count INT DEFAULT 0,
                topics TEXT,
                license VARCHAR(100),
                owner VARCHAR(255),
                description TEXT,
                is_fork BOOLEAN DEFAULT FALSE,
                collected_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                INDEX idx_full_name (full_name),
                INDEX idx_stargazers (stargazers_count),
                INDEX idx_language (language),
                INDEX idx_owner (owner)
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
            """
            cursor.execute(create_table_query)
            print("✅ 테이블 생성/확인 완료")
            
            # 데이터 삽입 쿼리 (중복 시 업데이트)
            insert_query = """
            INSERT INTO github_repos (
                id, full_name, stargazers_count, forks_count, language,
                created_at, updated_at, open_issues_count, topics,
                license, owner, description, is_fork
            ) VALUES (
                %(id)s, %(full_name)s, %(stargazers_count)s, %(forks_count)s,
                %(language)s, %(created_at)s, %(updated_at)s, %(open_issues_count)s,
                %(topics)s, %(license)s, %(owner)s, %(description)s, %(is_fork)s
            )
            ON DUPLICATE KEY UPDATE
                stargazers_count=VALUES(stargazers_count),
                forks_count=VALUES(forks_count),
                language=VALUES(language),
                updated_at=VALUES(updated_at),
                open_issues_count=VALUES(open_issues_count),
                topics=VALUES(topics),
                license=VALUES(license),
                description=VALUES(description),
                is_fork=VALUES(is_fork),
                collected_at=CURRENT_TIMESTAMP
            """
            
            inserted_count = 0
            updated_count = 0
            
            for repo in repos_list:
                # 데이터 추출 및 변환
                repo_data = {
                    'id': repo['id'],
                    'full_name': repo['full_name'],
                    'stargazers_count': repo['stargazers_count'],
                    'forks_count': repo['forks_count'],
                    'language': repo['language'],
                    'created_at': repo['created_at'],
                    'updated_at': repo['updated_at'],
                    'open_issues_count': repo['open_issues_count'],
                    'topics': ','.join(repo.get('topics', [])),  # 리스트를 문자열로 변환
                    'license': repo['license']['spdx_id'] if repo.get('license') else None,
                    'owner': repo['owner']['login'],
                    'description': repo['description'],
                    'is_fork': repo['fork']
                }
                
                # 데이터 삽입
                affected_rows = cursor.execute(insert_query, repo_data)
                
                if affected_rows == 1:
                    inserted_count += 1
                elif affected_rows == 2:
                    updated_count += 1
                
                if (inserted_count + updated_count) % 100 == 0:
                    print(f"진행중... {inserted_count + updated_count}개 처리 완료")
            
            connection.commit()
            print(f"\n✅ MySQL 저장 완료!")
            print(f"   - 신규 삽입: {inserted_count}개")
            print(f"   - 업데이트: {updated_count}개")
            print(f"   - 총 처리: {inserted_count + updated_count}개")
            
    except pymysql.Error as e:
        print(f"❌ MySQL 오류 발생: {e}")
        connection.rollback()
    except Exception as e:
        print(f"❌ 저장 중 오류 발생: {e}")
        connection.rollback()
    finally:
        connection.close()


In [ ]:
all_repos = []
for kw in keyword:
    print(f"Searching keyword: {kw}")
    result = fetch_repos(kw)
    all_repos.extend(result)

# full_name 기준 중복 제거
unique_repos = {repo["full_name"]: repo for repo in all_repos}
repos_list = list(unique_repos.values())
print(f"총 {len(repos_list)}개의 고유 레포 수집 완료")

# DataFrame 생성 및 CSV 저장
df = pd.DataFrame([get_repo_details(repo) for repo in repos_list])
df.to_csv('repos_data.csv', index=False, encoding='utf-8')
print("CSV 파일 저장 완료")

Searching keyword: robotics
Searching keyword: ROS
Searching keyword: robot arm
Searching keyword: robot
Searching keyword: amr
총 2148개의 고유 레포 수집 완료
CSV 파일 저장 완료


In [8]:
# MySQL에 저장
print("=" * 50)
print("MySQL 데이터베이스에 저장 시작")
print("=" * 50)
save_to_mysql(repos_list)

MySQL 데이터베이스에 저장 시작
✅ 테이블 생성/확인 완료
진행중... 100개 처리 완료
진행중... 200개 처리 완료
진행중... 300개 처리 완료
진행중... 400개 처리 완료
진행중... 500개 처리 완료
진행중... 600개 처리 완료
진행중... 700개 처리 완료
진행중... 800개 처리 완료
진행중... 900개 처리 완료
진행중... 1000개 처리 완료
진행중... 1100개 처리 완료
진행중... 1200개 처리 완료
진행중... 1300개 처리 완료
진행중... 1400개 처리 완료
진행중... 1500개 처리 완료
진행중... 1600개 처리 완료
진행중... 1700개 처리 완료
진행중... 1800개 처리 완료
진행중... 1900개 처리 완료
진행중... 2000개 처리 완료
진행중... 2100개 처리 완료

✅ MySQL 저장 완료!
   - 신규 삽입: 2148개
   - 업데이트: 0개
   - 총 처리: 2148개
